In [1]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import re
def read_pgm(filename, byteorder='>'):
    """Return image data from a raw PGM file as numpy array.

    Format specification: http://netpbm.sourceforge.net/doc/pgm.html

    """
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))



In [2]:
filename = "downgesture_train.list"
with open(filename, 'r') as f:
    my_list = [line.rstrip('\n') for line in f]
outputList=[1 if i.find('down')>0 else 0 for i in [x for x in my_list]]
outputList=np.array(outputList).reshape(184,1)
outputList.shape

(184, 1)

In [3]:
inputList=[]
tempL=[]
for k in range(len(my_list)):
    sampleImage=read_pgm(my_list[k],byteorder='<')
    sampleImage.setflags(write=1)
    sampleImage=sampleImage.reshape(960,1)
    tempL=np.zeros(960)
    for i in range(0,960):
        tempL[i]=(sampleImage[i]/255.0)
    #tempL[0]=1
    inputList.append(tempL.T)
inputList=np.array(inputList, dtype=np.float64)

In [4]:
clf = MLPClassifier(solver='sgd', alpha=0,
                  hidden_layer_sizes=(100,), activation='logistic', learning_rate_init=0.1,
                  max_iter=1000)

clf.fit(inputList,outputList)



C:\Users\aradh\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='logistic', alpha=0, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.1, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [5]:
filename = "downgesture_test.list"
with open(filename, 'r') as f:
    my_list = [line.rstrip('\n') for line in f]
testList=[1 if i.find('down')>0 else 0 for i in [x for x in my_list]]
testList=np.array(testList).reshape(83,1)


In [6]:
inputList=[]
tempL=[]
for k in range(len(my_list)):
    sampleImage=read_pgm(my_list[k],byteorder='<')
    sampleImage.setflags(write=1)
    sampleImage=sampleImage.reshape(960,1)
    tempL=np.zeros(960)
    for i in range(1,960):
        tempL[i]=(sampleImage[i]/255.0)
    
    inputList.append(tempL.T)
inputList=np.array(inputList)

In [7]:
pred_=clf.predict(inputList)
print(pred_)

[1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 1 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0]


### Accuracy

In [8]:
accuracy = (sum([True if pred_[i] ==testList[i] else False for i in range(len(pred_))]))*100/83
print (accuracy)

89.1566265060241
